In [1]:
library(tidyverse)
library(tidymodels)
library(cleandata)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   1.0.1 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.5.0 
✔ readr   2.1.3      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.2     ✔ rsample      1.1.1
✔ dials        1.1.0     ✔ tune         1.0.1
✔ infer        1.0.4     ✔ workflows    1.1.2
✔ modeldata    1.0.1     ✔ workflowsets 1.0.0
✔ parsnip      1.0.3     ✔ yardstick    1.1.0
✔ recipes      1.0.4     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() ma

In [2]:
data <- read.csv("data/amazon_beauty_data.csv")
head(data)

,X,main_cat,price,asin,overall
,<int>,<chr>,<chr>,<chr>,<dbl>
1,0,All Beauty,,6546546450,4
2,1,All Beauty,,6546546450,1
3,2,All Beauty,$44.99,7178680776,3
4,3,All Beauty,$28.76,7250468162,5
5,4,All Beauty,,7367905066,5
6,5,All Beauty,$12.15,7414204790,5


In [3]:
data <- data %>%
        mutate(main_cat = as.factor(main_cat))%>%
        mutate(price =substr(price, 2, nchar(price)))%>%
        mutate(price = as.numeric(price))%>%
        group_by(asin)%>%
        summarize(product_avg_ratings = mean(overall), price = price, main_cat = main_cat)
head(data)

Warning message in mask$eval_all_mutate(quo):
“NAs introduced by coercion”
`summarise()` has grouped output by 'asin'. You can override using the
`.groups` argument.


asin,product_avg_ratings,price,main_cat
<chr>,<dbl>,<dbl>,<fct>
6546546450,2.5,NA,All Beauty
6546546450,2.5,NA,All Beauty
7178680776,3.0,44.99,All Beauty
7250468162,5.0,28.76,All Beauty
7367905066,5.0,NA,All Beauty
7414204790,4.4,12.15,All Beauty


In [4]:
print(sum(is.na(data$price)))

[1] 304810


In [5]:
data <- data %>%
        drop_na()%>%
        distinct(asin, .keep_all=TRUE)
head(data)

asin,product_avg_ratings,price,main_cat
<chr>,<dbl>,<dbl>,<fct>
7178680776,3.00000,44.99,All Beauty
7250468162,5.00000,28.76,All Beauty
7414204790,4.40000,12.15,All Beauty
8921463194,1.00000,24.99,All Beauty
9752980953,1.00000,21.95,All Beauty
9790787006,4.22807,13.85,All Beauty


In [6]:
data['main_cat'] <- encode_binary(data.frame(data['main_cat']), full_print=FALSE)
head(data)

coded 1 cols 


asin,product_avg_ratings,price,main_cat
<chr>,<dbl>,<dbl>,<fct>
7178680776,3.00000,44.99,0
7250468162,5.00000,28.76,0
7414204790,4.40000,12.15,0
8921463194,1.00000,24.99,0
9752980953,1.00000,21.95,0
9790787006,4.22807,13.85,0


In [7]:
summary(data)

     asin           product_avg_ratings     price         main_cat 
 Length:18023       Min.   :1.000       Min.   :   0.01   0:11171  
 Class :character   1st Qu.:3.818       1st Qu.:  11.89   1: 6852  
 Mode  :character   Median :4.333       Median :  21.00            
                    Mean   :4.140       Mean   :  34.37            
                    3rd Qu.:5.000       3rd Qu.:  39.00            
                    Max.   :5.000       Max.   : 999.99            

In [8]:
sd(data$product_avg_ratings)

[1] 0.9255929

In [9]:
sd(data$price)

[1] 47.59821

In [10]:
# save(data, file = "dataset.RData")

In [11]:
write.csv(data, file = "data/cleaned_dataset.csv")